Pipeline to get the text on a webpage

In [ ]:
# Installing libraries
!pip install requests
!pip install beautifulsoup4
!pip install scikit-learn
!pip install transformers
!pip install bert-extractive-summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.4 MB/s eta 0:00:00


In [ ]:
# importing libraries

import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from urllib.parse import urlparse, urljoin
import json
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForQuestionAnswering
import random as rnd
from termcolor import colored, cprint
import os
import warnings
warnings.filterwarnings('ignore')
import torch
from summarizer import Summarizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# import concurrent.futures
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin

def get_weblinks(output, root_links, root):
    new = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(crawl_page, link, root, output, new) for link in root_links}
        concurrent.futures.wait(futures)

    if len(new)>0 or len(output)<100:
        output.extend(new)
        return get_weblinks(output, new, root)
    return output

def crawl_page(link, root, output, new):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')

    for a_tag in links:
        href = a_tag.get('href')
        if href:
            href = urljoin(link, href)
            if href not in output and href.startswith(root):
                new.append(href)

# # Sample
# url = ['https://www.mayoclinic.org/diseases-conditions/index?letter=A']
# root = 'https://www.mayoclinic.org'
# a = []
# final = get_weblinks(a, url, root)


In [ ]:
# getting links
# get all links on webpage and append all starting with root then loop through them to do the same
def get_weblinks(output, root_links, root):
    count = 0
    new = []
    for i in root_links:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')

        for link in links:
            href = link.get('href')
            if href is not None:
              if href.startswith('/'):
                href = urljoin(i, href)

                # if any(root_link in href for root_link in root_links) and href not in output:
                #     new.append(href)
                #     count += 1
              if href not in output:
                if href.startswith(root):
                  new.append(href)
                  count += 1
            else:
              continue

    output.extend(new)

    if count == 0 or len(set(output)) > 1000:
        return output
    else:
        return get_weblinks(output, new, root)

# sample
url = ['https://www.mayoclinic.org/diseases-conditions/index?letter=A']
# url = ['https://www.mayoclinic.org']
root = 'https://www.mayoclinic.org'
a = []
final = get_weblinks(a, url, root)

In [ ]:
# final

['https://www.mayoclinic.org/',
 'https://www.mayoclinic.org/patient-centered-care',
 'https://www.mayoclinic.org/about-mayo-clinic',
 'https://www.mayoclinic.org/appointments',
 'https://www.mayoclinic.org/appointments/find-a-doctor',
 'https://www.mayoclinic.org/locations',
 'https://www.mayoclinic.org/patient-visitor-guide',
 'https://www.mayoclinic.org/patient-visitor-guide/billing-insurance',
 'https://www.mayoclinic.org/departments-centers',
 'https://www.mayoclinic.org/departments-centers/international',
 'https://www.mayoclinic.org/about-mayo-clinic/contact',
 'https://www.mayoclinic.org/patient-visitor-guide',
 'https://www.mayoclinic.org/diseases-conditions',
 'https://www.mayoclinic.org/symptoms',
 'https://www.mayoclinic.org/tests-procedures',
 'https://www.mayoclinic.org/drugs-supplements',
 'https://www.mayoclinic.org/healthy-lifestyle',
 'https://www.mayoclinic.org/diseases-conditions',
 'https://www.mayoclinic.org/medical-professionals',
 'https://www.mayoclinic.org/med

In [ ]:
# len(final)

In [ ]:
def extract_text_from_url(url, class_name=['cmp-text__rich-content', 'p']):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        extracted_text = ''

        for i in class_name:    
            text_elements = soup.find_all(i)
            

            for text_element in text_elements:
                extracted_text += text_element.get_text()
                # You can print the extracted text here if needed

        return extracted_text

    except requests.exceptions.RequestException as e:
        print(f"Error while fetching data from {url}: {e}")
        return None

# sample
# Example usage:
# url = 'https://www.mayoclinic.org/diseases-conditions/malaria/symptoms-causes/syc-20351184'

# extracted_text = med_extract_text_from_url(url)

# if extracted_text:
#     print(extracted_text)
# else:
#     print("Failed to extract text from the URL.")

Malaria is a disease caused by a parasite. The parasite is spread to humans through the bites of infected mosquitoes. People who have malaria usually feel very sick with a high fever and shaking chills.While the disease is uncommon in temperate climates, malaria is still common in tropical and subtropical countries. Each year nearly 290 million people are infected with malaria, and more than 400,000 people die of the disease.To reduce malaria infections, world health programs distribute preventive drugs and insecticide-treated bed nets to protect people from mosquito bites. The World Health Organization has recommended a malaria vaccine for use in children who live in countries with high numbers of malaria cases.Protective clothing, bed nets and insecticides can protect you while traveling. You also can take preventive medicine before, during and after a trip to a high-risk area. Many malaria parasites have developed resistance to common drugs used to treat the disease.Signs and sympto

In [ ]:

# import requests
# from bs4 import BeautifulSoup

# def extract_text_from_url(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Check for any HTTP errors
#         soup = BeautifulSoup(response.text, 'html.parser')

#         # Function to recursively extract text from HTML elements
#         def extract_text_from_element(element):
#             if element.name in ['script', 'style']:
#                 return ''  # Exclude script and style tags
#             if isinstance(element, str):
#                 return element
#             result = ''
#             if element.name == 'abbr':
#                 result += element.get('title', '')  # Include text from the 'title' attribute of <abbr> tags
#             for content in element.contents:
#                 result += extract_text_from_element(content)
#             return result

#         # Extract text from the entire HTML structure
#         extracted_text = extract_text_from_element(soup)

#         return extracted_text

#     except requests.exceptions.RequestException as e:
#         print(f"Error while fetching data from {url}: {e}")
#         return None

# # Example usage
# webpage_url = "https://timesofindia.indiatimes.com/blogs/ruminations/virat-kohli-and-the-virtue-of-selfishness/"  # Replace with the URL of the webpage you want to scrape
# webpage_text = extract_text_from_url(webpage_url)

# if webpage_text:
#     # Print or process the extracted text
#     print(webpage_text)


In [ ]:
# if there are many links - try doing 100 and get 10 and store and keep on going in recursion until you get the last 10

In [ ]:
def preprocess_text(input_text):
    # Tokenization
    tokens = word_tokenize(input_text)

    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Remove special characters, numbers, and extra spaces
    cleaned_text = ' '.join([re.sub(r'[^a-zA-Z]', ' ', token) for token in lemmatized_tokens])

    return cleaned_text

# Example usage
# input_text = "Malaria is a disease caused by a parasite. The parasite is spread to humans through the bites of infected mosquitoes."
# cleaned_text = preprocess_text(input_text)
# print(cleaned_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_tfidf_similarity(links, question):

    documents = []
    for link in links:
      if 'mayo' in link:
        documents.append(med_extract_text_from_url(link))
      else:
        documents.append(sports_extract_text_from_url(link))

    # Add the question to the list of documents
    documents.append(question)

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the documents to TF-IDF vectors
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Calculate cosine similarity between the question vector and all document vectors
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Extract the cosine similarity scores
    cosine_similarities = cosine_similarities[0]

    # Create a list of tuples with document text and its corresponding similarity score
    document_similarity_scores = list(zip(links, cosine_similarities))

    # Sort the list by similarity score in descending order
    document_similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Return the sorted list of document text and their relative TF-IDF scores
    return document_similarity_scores


In [ ]:
# Summarize data on webpage
# final = result[0][0]

from transformers import BartForConditionalGeneration, BartTokenizer

def summarize_text(text, max_length=512):
    # Load the pre-trained BART model and tokenizer
    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    # Tokenize the input text and generate a summary
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=True)
    summary_ids = model.generate(input_ids, max_length=60, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary and return it as a string
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage
# text = extract_text_from_url(final)
# summary = summarize_text(text)

# print("Summary:")
# print(summary)


In [ ]:
# QnA bert model on select data


tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def question_answer(question, passage):
    # Tokenize the question and passage
    inputs = tokenizer.encode_plus(question, passage, add_special_tokens=True, return_tensors="pt", max_length=512, truncation=True)

    # Model output using input_ids and token_type_ids
    with torch.no_grad():
        output = model(input_ids=inputs['input_ids'], token_type_ids=inputs['token_type_ids'])

    # Reconstruct the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits) + 1  # Adjust for inclusive end index
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    if answer == "[CLS]":
        answer = "Unable to find the answer to your question."

    return answer




Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def final(question, links):

  results = compute_tfidf_similarity(links, question)

  context = extract_text_from_url(results[0][0])
  


  answer = question_answer(question, context)

  return answer

In [ ]:
# associate the dropdown selection of option with the links below. if medical is selected then these links will be considered
medical = ['https://www.mayoclinic.org/diseases-conditions/malaria/symptoms-causes/syc-20351184',
        'https://www.mayoclinic.org/diseases-conditions/achilles-tendinitis/symptoms-causes/syc-20369020',
        ]
sports = ['https://timesofindia.indiatimes.com/blogs/ruminations/virat-kohli-and-the-virtue-of-selfishness/',
          'https://timesofindia.indiatimes.com/blogs/Swaminomics/here-is-how-india-can-afford-to-host-2036-olympics/']


question  = input("Enter your Question: ")


# then pass the typed question and the selected domain through this function to get the answer.
final(question, sports)

Enter your Question: What did prime minister modi say last week?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


'\nindia will leave no stone unturned in its bid to host the 2036 olympic games'

In [ ]:
# sab ek saath

website = input("Please enter your link:\n")
question = input("Enter question:")

links = ['https://www.mayoclinic.org/diseases-conditions/malaria/symptoms-causes/syc-20351184',
        'https://www.mayoclinic.org/diseases-conditions/achilles-tendinitis/symptoms-causes/syc-20369020',
        ]

while True:

    results = compute_tfidf_similarity(links, question)

    context = extract_text_from_url(results[0][0])

    answer = question_answer(question, context)
    print("\n" + answer)

    response = input("\nDo you want to ask another question based on this text (Y/N)? ")
    if response[0].lower() == "y":
        question = input("\nPlease enter your question:\n")
    elif response[0].lower() == "n":
        print("\nBye!")
        break

Please enter your link:
www.mayoclinic.org
Enter question:What is malaria?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



a disease caused by a parasite

Do you want to ask another question based on this text (Y/N)? y

Please enter your question:
what is achilles tendonitis?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



an overuse injury

Do you want to ask another question based on this text (Y/N)? y

Please enter your question:
symptoms of achilles tendonitis?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



the pain associated with achilles tendinitis typically begins as a mild ache in the back of the leg or above the heel after running or other sports activity .

Do you want to ask another question based on this text (Y/N)? n

Bye!


In [ ]:
# trials
# doc2vec on select webpages
documents = ['https://www.mayoclinic.org/diseases-conditions/malaria/symptoms-causes/syc-20351184',
                'https://www.mayoclinic.org/diseases-conditions/tendinitis/symptoms-causes/syc-20378243',
                'https://www.mayoclinic.org/diseases-conditions/retinal-diseases/symptoms-causes/syc-20355825',
                'https://www.mayoclinic.org/diseases-conditions/achilles-tendinitis/symptoms-causes/syc-20369020',
                'https://www.mayoclinic.org/diseases-conditions/acl-injury/symptoms-causes/syc-20350738',
                'https://www.mayoclinic.org/diseases-conditions/acne/symptoms-causes/syc-20368047',
                'https://www.mayoclinic.org/diseases-conditions/hiv-aids/symptoms-causes/syc-20373524',
                'https://www.mayoclinic.org/diseases-conditions/acute-sinusitis/symptoms-causes/syc-20351671',
                'https://www.mayoclinic.org/diseases-conditions/allergies/symptoms-causes/syc-20351497']

for i in documents:
  print(extract_text_from_url(i))

Malaria is a disease caused by a parasite. The parasite is spread to humans through the bites of infected mosquitoes. People who have malaria usually feel very sick with a high fever and shaking chills. While the disease is uncommon in temperate climates, malaria is still common in tropical and subtropical countries. Each year nearly 290 million people are infected with malaria, and more than 400,000 people die of the disease. To reduce malaria infections, world health programs distribute preventive drugs and insecticide-treated bed nets to protect people from mosquito bites. The World Health Organization has recommended a malaria vaccine for use in children who live in countries with high numbers of malaria cases. Protective clothing, bed nets and insecticides can protect you while traveling. You also can take preventive medicine before, during and after a trip to a high-risk area. Many malaria parasites have developed resistance to common drugs used to treat the disease. Signs and sy